In [1]:
#!/usr/bin/env python

import numpy as np
import pandas as pd
import datetime as dt


In [2]:
def splitInToYears(df):
    cols = df.columns
    
    df2016 = pd.DataFrame(columns = cols)
    df2017 = pd.DataFrame(columns = cols)
    df2018 = pd.DataFrame(columns = cols)
    df2019 = pd.DataFrame(columns = cols)
    df2020 = pd.DataFrame(columns = cols)

    for index, row in df.iterrows():
        if dt.datetime.strptime(df['Date'][index],'%Y-%m-%d').year == 2016:
            df2016 = df2016.append(row,ignore_index=True)
        if dt.datetime.strptime(df['Date'][index],'%Y-%m-%d').year == 2017:
            df2017 = df2017.append(row,ignore_index=True)
        if dt.datetime.strptime(df['Date'][index],'%Y-%m-%d').year == 2018:
            df2018 = df2018.append(row,ignore_index=True)
        if dt.datetime.strptime(df['Date'][index],'%Y-%m-%d').year == 2019:
            df2019 = df2019.append(row,ignore_index=True)
        if dt.datetime.strptime(df['Date'][index],'%Y-%m-%d').year == 2020:
            df2020 = df2020.append(row,ignore_index=True)
    return [df2016, df2017,df2018, df2019,df2020]

In [3]:

## --- Data Wrangling ---

# Group A
VFIAX = pd.read_csv("Data/VFIAX.csv")
VFIAX.columns = ['Date','Open','High','Low','Close','VFIAX Close','Volume']
VBTLX = pd.read_csv("Data/VBTLX.csv")
VBTLX.columns = ['Date','Open','High','Low','Close','VBTLX Close','Volume']
VGSLX = pd.read_csv("Data/VGSLX.csv")
VGSLX.columns = ['Date','Open','High','Low','Close','VGSLX Close','Volume']

# Group B
VIMAX = pd.read_csv("Data/VIMAX.csv")
VIMAX.columns = ['Date','Open','High','Low','Close','VIMAX Close','Volume']
VSMAX = pd.read_csv("Data/VSMAX.csv")
VSMAX.columns = ['Date','Open','High','Low','Close','VSMAX Close','Volume']
VGHCX = pd.read_csv("Data/VGHCX.csv")
VGHCX.columns = ['Date','Open','High','Low','Close','VGHCX Close','Volume']

# Group C
AMZN = pd.read_csv("Data/AMZN.csv")
AMZN.columns = ['Date','Open','High','Low','Close','AMZN Close','Volume']
WMT = pd.read_csv("Data/WMT.csv")
WMT.columns = ['Date','Open','High','Low','Close','WMT Close','Volume']
CVS = pd.read_csv("Data/CVS.csv")
CVS.columns = ['Date','Open','High','Low','Close','CVS Close','Volume']


In [27]:

## --- Assemble -- code into a dataframe for Close of Day ---

close = pd.concat([VFIAX['Date'], VFIAX['VFIAX Close'], VBTLX['VBTLX Close'], VGSLX['VGSLX Close'], VIMAX['VIMAX Close'], VSMAX['VSMAX Close'], VGHCX['VGHCX Close'], AMZN['AMZN Close'], WMT['WMT Close'], CVS['CVS Close'] ], axis=1)
#print(close)

## --- generate mean daily return ---

dailyReturn = pd.DataFrame(columns = ['Date', 'VFIAX Daily Return','VBTLX Daily Return','VGSLX Daily Return', 'VIMAX Daily Return', 'VSMAX Daily Return', 'VGHCX Daily Return','AMZN Daily Return', 'WMT Daily Return','CVS Daily Return'])
for index, row in close.iterrows():
    if index == 0: continue
    #print((close['VFIAX Close'][index] - close['VFIAX Close'][index-1])/ (close['VFIAX Close'][index-1]))
    dailyReturn = dailyReturn.append({'Date': close['Date'][index],
                'VFIAX Daily Return': ((close['VFIAX Close'][index] - close['VFIAX Close'][index-1])/(close['VFIAX Close'][index-1])),
                'VBTLX Daily Return': ((close['VBTLX Close'][index] - close['VBTLX Close'][index-1])/(close['VBTLX Close'][index-1])),
                'VGSLX Daily Return': ((close['VGSLX Close'][index] - close['VGSLX Close'][index-1])/(close['VGSLX Close'][index-1])),
                'VIMAX Daily Return': ((close['VIMAX Close'][index] - close['VIMAX Close'][index-1])/(close['VIMAX Close'][index-1])),
                'VSMAX Daily Return': ((close['VSMAX Close'][index] - close['VSMAX Close'][index-1])/(close['VSMAX Close'][index-1])),
                'VGHCX Daily Return': ((close['VGHCX Close'][index] - close['VGHCX Close'][index-1])/(close['VGHCX Close'][index-1])),
                'AMZN Daily Return': ((close['AMZN Close'][index] - close['AMZN Close'][index-1])/(close['AMZN Close'][index-1])),
                'WMT Daily Return': ((close['WMT Close'][index] - close['WMT Close'][index-1])/(close['WMT Close'][index-1])),
                'CVS Daily Return': ((close['CVS Close'][index] - close['CVS Close'][index-1])/(close['CVS Close'][index-1]))},ignore_index=True)



In [28]:
dailyReturnsOverTimespan = splitInToYears(dailyReturn)

In [29]:
## --- Estimator dataframes ---
# returns array of dataframes split by year in accending order
def calculateEstimators(dataframes):
    estimators = []
    assets = ['VFIAX', 'VBTLX', 'VGSLX', 'VIMAX', 'VSMAX', 'VGHCX', 'AMZN', 'WMT', 'CVS']
    for df in dataframes:
        data = df[df.columns[1:]]
        uniformWeights = 1/data.shape[0]
        
        wBar = np.sum(np.square(np.ones(data.shape[0]) * uniformWeights))
        
        mean = np.array(np.sum(data, axis=0) * uniformWeights).reshape(-1,1)
        difference = np.subtract(data , np.matmul(mean,np.ones((data.shape[0],1)).transpose()).transpose())
        variance =  np.array(1/(1 - wBar) * np.sum(uniformWeights * np.square(difference), axis = 0)).reshape(-1,1)
        StdOfExpectedValue = np.array(np.sqrt(wBar) * np.sqrt(variance)).reshape(-1,1)
        
        signalToNoise = np.array(mean/StdOfExpectedValue).reshape(-1,1)
        estimator = pd.DataFrame({
                    'Expected Return': mean.reshape(-1,),
                    'Variance Estimator': variance.reshape(-1,),
                    'Standard Dev of Expected Return': StdOfExpectedValue.reshape(-1,),
                    'Signal to Noise': signalToNoise.reshape(-1,)},
                    index=assets)
        estimators.append(estimator)
    return estimators  
# print(ExpectedReturn)


In [30]:

estimators = calculateEstimators(dailyReturnsOverTimespan)


In [31]:
for years_after_2016, estimator in enumerate(estimators):
    print('\n' + str(2016 + years_after_2016) + '\n', estimator.sort_values(by='Signal to Noise', axis=0, ascending=False))


2016
        Expected Return  Variance Estimator  Standard Dev of Expected Return  \
VSMAX         0.000723            0.000112                         0.000666   
VFIAX         0.000482            0.000068                         0.000520   
WMT           0.000663            0.000147                         0.000764   
VIMAX         0.000468            0.000092                         0.000605   
VBTLX         0.000103            0.000005                         0.000142   
VGSLX         0.000380            0.000115                         0.000676   
AMZN          0.000586            0.000350                         0.001179   
CVS          -0.000682            0.000190                         0.000867   
VGHCX        -0.000606            0.000111                         0.000664   

       Signal to Noise  
VSMAX         1.085166  
VFIAX         0.926836  
WMT           0.866886  
VIMAX         0.773408  
VBTLX         0.726289  
VGSLX         0.562157  
AMZN          0.497363  
CV

In [32]:
import math
logReturns = dailyReturn[dailyReturn.columns[1:]].applymap(math.log1p)
logReturns.insert(0,'Date',dailyReturn[dailyReturn.columns[0]])

logReturnsOverTimeSpan = splitInToYears(logReturns)

estimators = calculateEstimators(logReturnsOverTimeSpan)

In [33]:
for years_after_2016, estimator in enumerate(estimators):
    print('\n' + str(2016+years_after_2016) + '\n', estimator.sort_values(by='Signal to Noise', ascending=False))


2016
        Expected Return  Variance Estimator  Standard Dev of Expected Return  \
VSMAX         0.000667            0.000112                         0.000667   
VFIAX         0.000448            0.000068                         0.000520   
WMT           0.000590            0.000145                         0.000758   
VBTLX         0.000100            0.000005                         0.000142   
VIMAX         0.000422            0.000093                         0.000607   
VGSLX         0.000323            0.000116                         0.000677   
AMZN          0.000412            0.000349                         0.001177   
CVS          -0.000779            0.000197                         0.000884   
VGHCX        -0.000661            0.000111                         0.000664   

       Signal to Noise  
VSMAX         0.999493  
VFIAX         0.860266  
WMT           0.778840  
VBTLX         0.708404  
VIMAX         0.695569  
VGSLX         0.476212  
AMZN          0.350242  
CV

In [ ]:
# ## --- split into years 2015 to 2020 ---

# ER2016 = pd.DataFrame(columns = ['Date', 'ER'])
# ER2017 = pd.DataFrame(columns = ['Date', 'ER'])
# ER2018 = pd.DataFrame(columns = ['Date', 'ER'])
# ER2019 = pd.DataFrame(columns = ['Date', 'ER'])
# ER2020 = pd.DataFrame(columns = ['Date', 'ER'])

# for index, row in ExpectedReturn.iterrows():
#     if dt.datetime.strptime(ExpectedReturn['Date'][index],'%Y-%m-%d').year == 2016:
#         ER2016 = ER2016.append({'Date':ExpectedReturn['Date'][index],'ER':ExpectedReturn['ER'][index]},ignore_index=True)
#     if dt.datetime.strptime(ExpectedReturn['Date'][index],'%Y-%m-%d').year == 2017:
#         ER2017 = ER2017.append({'Date':ExpectedReturn['Date'][index],'ER':ExpectedReturn['ER'][index]},ignore_index=True)
#     if dt.datetime.strptime(ExpectedReturn['Date'][index],'%Y-%m-%d').year == 2018:
#         ER2018 = ER2018.append({'Date':ExpectedReturn['Date'][index],'ER':ExpectedReturn['ER'][index]},ignore_index=True)
#     if dt.datetime.strptime(ExpectedReturn['Date'][index],'%Y-%m-%d').year == 2019:
#         ER2019 = ER2019.append({'Date':ExpectedReturn['Date'][index],'ER':ExpectedReturn['ER'][index]},ignore_index=True)
#     if dt.datetime.strptime(ExpectedReturn['Date'][index],'%Y-%m-%d').year == 2020:
#         ER2020 = ER2020.append({'Date':ExpectedReturn['Date'][index],'ER':ExpectedReturn['ER'][index]},ignore_index=True)

# print("2016: ")
# print(ER2016)
# print("2017: ")
# print(ER2017)
# print("2018: ")
# print(ER2018)
# print("2019: ")
# print(ER2019)
# print("2020: ")
# print(ER2020)